# Agentic AI Workshop — Hands-on Jupyter Notebook
This notebook is structured for a **2-hour interactive workshop** covering:

1. Simple LLM calling
2. Prompt engineering
3. Tools (safe helper functions)
4. Simple LangChain usage
5. Types of Chains (Conversation, Summary, etc.)
6. LangChain with Memory
7. Retrieval-Augmented Generation (RAG)

**How to use:** Run cells sequentially. If a cell fails due to missing packages, follow the error message to `pip install` the required package(s).

Replace `.env` values (e.g., `OPENAI_API_KEY`) before running cells that call external APIs. There are local alternatives where possible (Chroma local vector store).

-----
Instructor notes: This notebook is "teaching-style" but written with clean, reusable functions so you can also show industry best practices.


## 0 — Setup & Environment
Load environment variables, check Python packages, and provide quick helper to install missing packages.

Make sure you have a `.env` file in the repo root with at least:

```
OPENAI_API_KEY=sk-...
```

If you will use Anthropic or other providers, add those keys similarly to `.env`.


In [1]:
# Utility: helpful installer for common packages used in this notebook.
import sys, subprocess, pkgutil

def ensure(package):
    try:
        __import__(package)
    except Exception:
        print(f"Installing {package} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    finally:
        print(f"{package} available.")

# Common packages we will use
packages = [
    ("dotenv", "python-dotenv"),
    ("openai", "openai"),
    ("langchain", "langchain"),
    ("chromadb", "chromadb"),
    ("tiktoken", "tiktoken"),
]

for mod, pkg in packages:
    try:
        __import__(mod)
    except Exception:
        print(f"Package {mod} not found. To install run: pip install {pkg}")
print("If packages are missing, run: pip install python-dotenv openai langchain chromadb tiktoken")

Package tiktoken not found. To install run: pip install tiktoken
If packages are missing, run: pip install python-dotenv openai langchain chromadb tiktoken


In [3]:
# Load environment variables
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env if present

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("WARNING: OPENAI_API_KEY not found in environment. Cells using OpenAI will fail until you set it.")
else:
    print("✅ OPENAI_API_KEY found.")

# Show which Python executable is used (useful in labs)
import sys
print("Python:", sys.executable)
print("Working dir:", os.getcwd())

✅ OPENAI_API_KEY found.
Python: /home/wajihailyas/Documents/personal/Agentic_AI_Workshop/.venv/bin/python
Working dir: /home/wajihailyas/Documents/personal/Agentic_AI_Workshop/notebooks


## 1 — Simple LLM Call
Minimal example calling an LLM (OpenAI). The code below is written defensively so you can switch providers.

This cell uses the `openai` Python client. If you prefer LangChain wrapper or Anthropic, we'll show variations later.

In [4]:
# Simple LLM call using OpenAI's `openai` package (modern usage with `OpenAI` client)
try:
    from openai import OpenAI
except Exception as e:
    raise RuntimeError("Please install the OpenAI Python client (pip install openai). Error: " + str(e))

import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise EnvironmentError('Set OPENAI_API_KEY in .env or environment to run this cell.')

client = OpenAI(api_key=OPENAI_API_KEY)

def simple_llm_call(
    prompt: str,
    model: str = "gpt-4o-mini",
    max_tokens: int = 200,
    temperature: float = 0.2,
) -> str:
    """
    Make a single LLM (chat completion) call and return the generated text.

    Args:
        prompt (str): The user prompt or instruction.
        model (str): Model name to use (default: gpt-4o-mini).
        max_tokens (int): Maximum number of tokens to generate.
        temperature (float): Sampling temperature (lower = more deterministic).

    Returns:
        str: The LLM's generated response text.
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"❌ Error during LLM call: {e}"

# Example usage
prompt = "Explain Agentic AI in 3 concise bullet points."
print("🧠 Prompt:\n", prompt, "\n\n💬 Model Output:\n")
print(simple_llm_call(prompt))

🧠 Prompt:
 Explain Agentic AI in 3 concise bullet points. 

💬 Model Output:

- **Definition**: Agentic AI refers to artificial intelligence systems that possess the ability to act autonomously, make decisions, and perform tasks without human intervention, often exhibiting goal-directed behavior.

- **Capabilities**: These systems can learn from their environment, adapt to new information, and optimize their actions to achieve specific objectives, often leveraging advanced machine learning techniques.

- **Applications**: Agentic AI is utilized in various fields, including robotics, autonomous vehicles, and intelligent personal assistants, where it enhances efficiency, decision-making, and operational effectiveness.


## 2 — Prompt Engineering Basics
We will demonstrate how prompt structure affects outputs. We'll show:
- A naive prompt
- A structured prompt with system + user roles
- A prompt with constraints and examples (few-shot)


In [5]:
# Demonstrate prompt variations using the helper `simple_llm_call`.
naive = "Summarize the following text: 'Agentic AI allows agents to use tools.'"
structured = (
    "System: You are an assistant that writes concise summaries.\n"
    "User: Summarize the following text in 1 sentence and provide 3 keywords.\n"
    "Text: 'Agentic AI allows agents to use tools.'"
)
few_shot = (
    "System: You are a helpful summarizer.\n"
    "User: Example:\n- Text: 'RAG integrates retrieval and LLMs.'\n- Summary: 'RAG combines retrieval with generation to ground answers.'\n- Keywords: retrieval, grounding, LLM\\n\n"
    "Now summarize:\nText: 'Agentic AI allows agents to use tools and memory to act.'\n"
)

print('=== Naive prompt output ===')
print(simple_llm_call(naive))
print('\n=== Structured prompt output ===')
print(simple_llm_call(structured))
print('\n=== Few-shot prompt output ===')
print(simple_llm_call(few_shot))


=== Naive prompt output ===
Agentic AI enables agents to utilize tools.

=== Structured prompt output ===
Agentic AI enables agents to utilize various tools.  
Keywords: Agentic AI, agents, tools.

=== Few-shot prompt output ===
Summary: 'Agentic AI enables agents to utilize tools and memory for action.'  
Keywords: agentic AI, tools, memory, action


## 3 — Tools: Creating Safe Helper Functions
Tools are functions the agent can call. We'll create several safe tools:
- Calculator (safe eval)
- Simple text summarizer using the LLM
- Sentiment checker (toy)

**Teaching note:** Never `eval` untrusted strings in production. Use parsers or safe expression evaluators. Here it's for demonstration in an isolated environment.

In [6]:
# Define some safe helper tools as pure Python functions.
from typing import Tuple

def safe_arith(expr: str) -> str:
    """Evaluate simple arithmetic expressions. Only allow digits and +-*/(). spaces.
    This is a *demonstration* safe parser; it's simplistic.
    """
    import re
    if not re.match(r"^[0-9\s+\-*/().]+$", expr):
        return "error: expression contains illegal characters"
    try:
        # using eval with restricted builtins for demo only
        result = eval(expr, {"__builtins__": None}, {})
        return str(result)
    except Exception as e:
        return f"error: {e}"


def summarizer(text: str, max_len: int = 80) -> str:
    """Simple LLM-based summarizer wrapper."""
    prompt = f"Summarize the following text in one short sentence (<= {max_len} chars):\n\n{text}"
    return simple_llm_call(prompt, max_tokens=150)


def sentiment_toy(text: str) -> str:
    """Very small toy sentiment function using keywords (not an LLM)"""
    t = text.lower()
    if any(w in t for w in ["good","great","love","excellent","happy"]):
        return "positive"
    if any(w in t for w in ["bad","terrible","hate","awful","sad"]):
        return "negative"
    return "neutral"

# Quick demo of the tools
print('calc 123*7 ->', safe_arith('123*7'))
print('summary ->', summarizer('Agentic AI systems can call tools and retrieve info to act more effectively.'))
print('sentiment ->', sentiment_toy('I love this workshop!'))

calc 123*7 -> 861
summary -> Agentic AI systems use tools and information to enhance their effectiveness.
sentiment -> positive


## 4 — Simple LangChain (LLMChain)
LangChain wraps LLMs with prompt templates. We'll show `LLMChain` as a building block.
Install `langchain` and try the example below. If LangChain version mismatch occurs, pin a compatible version in `requirements.txt`.


In [21]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv
import os

# Load environment
load_dotenv()

# Initialize Groq LLM
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile", 
    temperature=0.7
)

# Create a prompt
template = """
You are an expert AI assistant.
Answer clearly and concisely.

User: {question}
"""
prompt = PromptTemplate(input_variables=["question"], template=template)

# Create a LangChain chain
chain = LLMChain(prompt=prompt, llm=llm)

# Run the chain
response = chain.run({"question": "What is artificial intelligence?"})
print(response)

Artificial intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception.


## 5 — Types of Chains in LangChain
We'll demonstrate several chain types with small examples:
1. LLMChain (done above)
2. ConversationChain (simple chat memory)
3. SequentialChain (run multiple chains in sequence)
4. MapReduceDocumentsChain (summarization pattern)
5. RetrievalQA (RAG preview)

Note: Some chain classes may require additional packages; the notebook uses simple implementations or local fallbacks.

In [23]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Use the default memory_key='history'
memory = ConversationBufferMemory(memory_key='history', return_messages=False)

# Initialize the chain
conv_chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Run conversation
print(conv_chain.run("Hi, who are you?"))
print(conv_chain.run("What did I just ask you?"))




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, who are you?
AI:

> Finished chain.
Hello. I'm an artificial intelligence language model, which means I'm a computer program designed to understand and generate human-like text. I was trained on a massive dataset of text from the internet, books, and other sources, which allows me to learn patterns and relationships in language. My training data includes a wide range of topics, from science and history to entertainment and culture. I'm a bit like a super-knowledgeable librarian, but instead of just recommending books, I can have conversations and answer questions to the best of my ability. I'm constantly learning and improving, and I'm excited to chat with 

In [25]:
# Interactive input loop
while True:
    user_input = input("🧑 You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    # Generate response using conversation chain
    response = conv_chain.run(user_input)
    print("🤖 AI:", response, "\n")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, who are you?
AI: Hello. I'm an artificial intelligence language model, which means I'm a computer program designed to understand and generate human-like text. I was trained on a massive dataset of text from the internet, books, and other sources, which allows me to learn patterns and relationships in language. My training data includes a wide range of topics, from science and history to entertainment and culture. I'm a bit like a super-knowledgeable librarian, but instead of just recommending books, I can have conversations and answer questions to the best of my ability. I'm constantly learning and improving, and I'm excited to chat with you. By the way, my 

In [26]:
# SequentialChain example: run two simple chains in sequence
from langchain.chains import SequentialChain
from langchain import PromptTemplate

# Small chain1: produce a topic from a seed
p1 = PromptTemplate(input_variables=['seed'], template='Provide a concise topic title for: {seed}')
c1 = LLMChain(llm=llm, prompt=p1, output_key='topic')

# Small chain2: expand the topic into a one-sentence description
p2 = PromptTemplate(input_variables=['topic'], template='Write a one-sentence description for the topic: {topic}')
c2 = LLMChain(llm=llm, prompt=p2, output_key='description')

seq = SequentialChain(chains=[c1, c2], input_variables=['seed'], output_variables=['topic','description'], verbose=True)

print(seq({'seed': 'agentic AI workshop'}))

/tmp/ipykernel_99046/958600381.py:15: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(seq({'seed': 'agentic AI workshop'}))




> Entering new SequentialChain chain...

> Finished chain.
{'seed': 'agentic AI workshop', 'topic': '"Empowering Autonomous Decision-Making: Agentic AI Workshop"', 'description': 'The "Empowering Autonomous Decision-Making: Agentic AI Workshop" is an innovative forum where experts and researchers gather to explore and develop artificial intelligence systems that can make autonomous decisions, fostering a new era of agentic AI that can learn, adapt, and interact with its environment in a more human-like manner.'}


In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


# --- Step 1: Create a long Q&A-style text ---
qa_text = """
Q: What is Agentic AI?
A: Agentic AI refers to systems that can take autonomous actions to achieve goals.

Q: How is it different from regular AI models?
A: Regular AI models only respond to inputs. Agentic AI can plan, decide, and use tools to complete tasks.

Q: What are the components of an agentic system?
A: Typically, these include an LLM, memory, tools, and a planner or controller.

Q: Where is Agentic AI used?
A: In personal assistants, autonomous research systems, and workflow automation.
"""

# --- Step 2: Split the text into chunks ---
splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=30)
chunks = splitter.split_text(qa_text)

print(f"📝 Split into {len(chunks)} chunks")

# --- Step 3: Create summarization prompt ---
summarize_prompt = ChatPromptTemplate.from_template(
    """Summarize the following text concisely, capturing the main points:

{text}

Summary:"""
)

# --- Step 4: Method 1 - Direct summarization (for small texts) ---
if len(chunks) <= 1:
    # Direct summarization
    chain = summarize_prompt | llm | StrOutputParser()
    summary = chain.invoke({"text": qa_text})
else:
    # --- Method 2 - Map-Reduce style (for larger texts) ---
    
    # Step 4a: Summarize each chunk
    chunk_summaries = []
    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}...")
        chain = summarize_prompt | llm | StrOutputParser()
        chunk_summary = chain.invoke({"text": chunk})
        chunk_summaries.append(chunk_summary)
    
    # Step 4b: Combine all chunk summaries
    combined_summaries = "\n\n".join(chunk_summaries)
    
    # Step 4c: Final summary
    final_prompt = ChatPromptTemplate.from_template(
        """The following are summaries of different parts of a document. 
Create a final concise summary that captures all the main points:

{summaries}

Final Summary:"""
    )
    chain = final_prompt | llm | StrOutputParser()
    summary = chain.invoke({"summaries": combined_summaries})

# --- Step 5: Display the summary ---
print("\n" + "="*60)
print("🧾 Original text length:", len(qa_text), "characters")
print("\n🤖 Summarized output:\n")
print(summary)
print("="*60)

📝 Split into 3 chunks
Processing chunk 1/3...
Processing chunk 2/3...
Processing chunk 3/3...

🧾 Original text length: 503 characters

🤖 Summarized output:

Here is a final concise summary that captures the main points:

Agentic AI is a type of autonomous AI that can plan, decide, and use tools to achieve specific goals, differing from regular AI models. It consists of four main components: a Large Language Model, memory, tools, and a planner/controller, and is applied in areas such as personal assistants, autonomous research, and workflow automation.


## 7 — RAG (Retrieval-Augmented Generation) with Chroma (local)
We will:
1. Create a local Chroma collection
2. Create embeddings for small documents using LangChain/OpenAI embeddings
3. Query the collection and show a RAG-style answer

This demo is intentionally small and local so it works in lab VMs without cloud vector DBs.


In [39]:
# RAG demo using chromadb + FREE sentence-transformers embeddings
try:
    import chromadb
    from langchain_community.embeddings import HuggingFaceEmbeddings
    from langchain_community.vectorstores import Chroma
    from langchain.chains import RetrievalQA
except Exception as e:
    print('If you see import errors, install: pip install chromadb langchain-community sentence-transformers')
    raise

# Prepare documents
docs = [
    "LangChain is a Python framework for developing applications powered by LLMs.",
    "RAG stands for Retrieval Augmented Generation: you retrieve relevant docs then generate grounded answers.",
    "Agents can use tools and memory to act autonomously."
]

print("🔄 Loading embedding model (this may take a moment on first run)...")

# Create FREE embeddings using sentence-transformers
# Popular models:
# - "all-MiniLM-L6-v2" (lightweight, fast, 384 dimensions)
# - "all-mpnet-base-v2" (better quality, 768 dimensions)
# - "paraphrase-multilingual-MiniLM-L12-v2" (multilingual support)

emb = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",  # Free, fast, good quality
    model_kwargs={'device': 'cpu'},  # Use 'cuda' if you have GPU
    encode_kwargs={'normalize_embeddings': True}  # Improves retrieval
)

print("✅ Embedding model loaded!")

# Create local Chroma vector store
print("📚 Creating vector store...")
vect = Chroma.from_texts(
    docs, 
    embedding=emb, 
    collection_name='workshop_demo',
    persist_directory="./chroma_db"  # Optional: persist to disk
)

print("✅ Vector store created!")

# Create a RetrievalQA chain
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type='stuff',  # 'stuff', 'map_reduce', 'refine', or 'map_rerank'
    retriever=vect.as_retriever(search_kwargs={"k": 2}),  # Retrieve top 2 docs
    return_source_documents=True  # Optional: return source docs
)

# Query the system
query = 'What is RAG?'
print('\n' + '='*60)
print('🔍 Query:', query)
print('='*60)

result = qa({"query": query})

print('\n🤖 Answer:')
print(result['result'])

# Show source documents (optional)
if 'source_documents' in result:
    print('\n📄 Source Documents:')
    for i, doc in enumerate(result['source_documents'], 1):
        print(f"{i}. {doc.page_content}")

print('='*60)

🔄 Loading embedding model (this may take a moment on first run)...


/home/wajihailyas/Documents/personal/Agentic_AI_Workshop/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Embedding model loaded!
📚 Creating vector store...
✅ Vector store created!

🔍 Query: What is RAG?

🤖 Answer:
RAG stands for Retrieval Augmented Generation. It involves retrieving relevant documents and then generating grounded answers based on them.

📄 Source Documents:
1. RAG stands for Retrieval Augmented Generation: you retrieve relevant docs then generate grounded answers.
2. LangChain is a Python framework for developing applications powered by LLMs.


In [41]:
# Compare model answer without RAG (direct LLM) vs with RAG (retrieval-augmented)
q = 'Explain Retrieval Augmented Generation (RAG) in one short paragraph.'
print('--- Without RAG ---\n')
print(simple_llm_call(q))
print('\n--- With RAG (retriever) ---\n')
result = qa({"query": q})
print(result['result'])

--- Without RAG ---

Retrieval Augmented Generation (RAG) is a machine learning framework that combines the strengths of information retrieval and natural language generation. It works by first retrieving relevant documents or pieces of information from a large corpus based on a given query, and then using that retrieved information to generate coherent and contextually relevant responses. This approach enhances the model's ability to produce accurate and informative outputs by grounding its responses in real-world data, thereby improving performance on tasks that require up-to-date knowledge or specific details.

--- With RAG (retriever) ---

Retrieval Augmented Generation (RAG) is a process where relevant documents are retrieved and then used to generate grounded answers. This approach combines the steps of retrieving relevant information and generating a response based on that information, allowing for more accurate and informed answers.


## 8 — Wrap-up, Exercises & Roadmap
**Exercises for students (pick 1-2):**

1. Add a new tool (e.g., a web search wrapper using SerpAPI) and integrate it into an agent.
2. Build a multi-step SequentialChain that generates a topic, expands it, then summarizes.
3. Improve the RAG dataset: add 10 short docs, tune `n_results`, and compare answers.

**Roadmap recap:** Python → ML fundamentals → LLM skills → Agent design → Infra & safety → portfolio projects.

Good luck! You can edit this notebook to add or remove cells before the workshop. 